# Размещение баннеров

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к нам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Нам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с нами сотрудничать.

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
import sklearn

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

In [4]:
data = pd.read_csv("checkins.csv", sep='|',skipinitialspace=True, skiprows=[1])

In [5]:
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984301,2041916.0,5222.0,NaN,NaN,2012-04-21 17:39:01
1,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
2,984315,1764391.0,5222.0,NaN,NaN,2012-04-21 17:37:18
3,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
4,984249,2146840.0,5222.0,NaN,NaN,2012-04-21 17:42:58


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1021967 entries, 0 to 1021966
Data columns (total 6 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   id                1021967 non-null  object 
 1   user_id           1021966 non-null  float64
 2   venue_id          1021966 non-null  float64
 3   latitude          396634 non-null   float64
 4   longitude         396634 non-null   float64
 5   created_at        1021966 non-null  object 
dtypes: float64(4), object(2)
memory usage: 46.8+ MB


In [7]:
data.dropna(how='any', inplace=True)

In [8]:
data

,id,user_id,venue_id,latitude,longitude,created_at
1,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
3,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
7,984291,105054.0,5222.0,45.523452,-122.676207,2012-04-21 17:39:22
9,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
10,984232,93870.0,380645.0,33.448377,-112.074037,2012-04-21 17:38:18
...,...,...,...,...,...,...
1021959,955561,626076.0,20073.0,40.850100,-73.866246,2012-04-13 09:56:48
1021960,955892,674797.0,2297.0,33.748995,-84.387982,2012-04-13 10:56:03
1021961,956377,845102.0,11195.0,42.765366,-71.467566,2012-04-13 12:08:45
1021962,956119,1139114.0,29488.0,42.439479,-83.743830,2012-04-13 11:36:44


In [9]:
sliced_data = data.iloc[:100000, 3:5]
sliced_data.shape

(100000, 2)

Используем MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

In [10]:
from sklearn.cluster import MeanShift

In [11]:
clustering = MeanShift(bandwidth=0.1)

In [12]:
cl = clustering.fit(sliced_data)

In [13]:
labels = cl.labels_

In [14]:
cluster_centers = cl.cluster_centers_
cluster_centers

array([[  40.7177164 ,  -73.99183542],
       [  33.44943805, -112.00213969],
       [  33.44638027, -111.90188756],
       ...,
       [ -37.8229826 ,  145.1811902 ],
       [ -41.2924945 ,  174.7732353 ],
       [ -45.0311622 ,  168.6626435 ]])

In [15]:
print('Количество выбранных кластеров : %d' % len(cluster_centers))

Количество выбранных кластеров : 3231


In [16]:
unique_labels, labels_counts = np.unique(labels, return_counts=True)

In [17]:
labels_more15 = list()
for index, count in enumerate(labels_counts):
    if count > 15:
        labels_more15.append(unique_labels[index])
print('Количество кластеров с числом элементов больше 15: %d' % len(labels_more15))

Количество кластеров с числом элементов больше 15: 592


In [18]:
cluster_centers_more15 = np.empty((len(labels_more15), 2))
for index, label in enumerate(labels_more15):
    cluster_centers_more15[index] = cluster_centers[label]
print(cluster_centers_more15.shape)

(592, 2)


In [19]:
offices = pd.read_csv('offices.csv')

In [20]:
def distance(x, y):
    return (np.sqrt(np.sum (x - y) ** 2))

In [21]:
distances_from_nearest_office = np.empty(cluster_centers_more15.shape[0])
for index, center in enumerate(cluster_centers_more15):
    min_distance = distance(center, offices.loc[0])
    for j in range(len(offices)):
        dist = distance(center, offices.loc[j])
        if dist < min_distance:
            min_distance = dist
    distances_from_nearest_office[index] = min_distance

sort_ind = np.argsort(distances_from_nearest_office)
final_dist = distances_from_nearest_office[sort_ind]
final_coord = cluster_centers_more15[sort_ind]

In [22]:
print('Минимальная дистанция:',final_dist[0])
print(final_coord[0])

Минимальная дистанция: 0.0013032129032541917
[52.37296399  4.89231722]
